## FUTURE CHANGES
* Remove test_train_split because this needs to be done before segmenting the datasets

In [4]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

DATASET_PATH = "data_static_60_5.json"

# load data
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
    
    # convert lists into numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
    #print(inputs[:1])
    
    return inputs, targets

if __name__ == "__main__":
   
    # load data
    inputs, targets = load_data(DATASET_PATH)
    print(inputs.shape)
    print(targets.shape)
    
    # split the data into train and test sets
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.3)  # 30% used for test set, rest for train set
                                                  
    
    # build CNN network architecture
    model = models.Sequential()
    model.add(layers.Conv2D(512, (3, 3), activation='relu', input_shape=(inputs.shape[1], inputs.shape[2], 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    

    '''    
    # build the network architecture
    model = keras.Sequential([
       
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])), 
            # take multidimensional array and flattens it out
            # remember that each track has many MFCC vectors at each hop length
            # first dimension is the interval, # second dimension is the amount of MFCCs
            # note: inputs is 3D array and [0] is the different segments, so we ignore
        
        # 1st hidden layer
        keras.layers.Conv2D(512, (3,3), activation="relu", input_shape=(32,32,3)),
        
        # 2nd hidden layer
        keras.layers.Dense(256, activation="relu"),
        
        # 3rd hidden layer
        keras.layers.Dense(64, activation="relu"),
        
        # output layer 
        # 5 neurons because predicting 5 categories clacky ,....
        keras.layers.Dense(5, activation="softmax")
    ])
    '''
    
    # compile network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                 loss="sparse_categorical_crossentropy",
                 metrics=['accuracy'])
    
    model.summary()
    
    # train network
    model.fit(inputs_train, 
              targets_train, 
              validation_data=(inputs_test, targets_test),
              epochs=50,
              batch_size=32) # important! # of samples in each of our batches
    
    
    
    
    
    
    

(2336, 22, 13)
(2336,)


ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node conv2d_3/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_3/Conv2D/ReadVariableOp)' with input shapes: [?,4,1,256], [3,3,256,64].